In [ ]:
! pip install spektral

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/path_to_your_folder/')

import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from spektral.layers import GATConv, GlobalAttentionPool

import scipy.io as sio
################################################################################
# PARAMETERS
################################################################################
l2_reg = 5e-4            # Regularization rate for l2
learning_rate = 1e-3     # Learning rate for Adam
epochs = 20              # Number of training epochs
batch_size = 32          # Batch size
es_patience = 20         # Patience for early stopping
K = 10 # Rician K factor
N_list = [16, 32, 64]    # Number of RIS elements
M_list = [16]            # Number of pilot symbols
max_iter = 20
################################################################################
# LOAD DATA
################################################################################


for N in N_list:
  for M in M_list:

    NMSE_h = np.empty((max_iter, 21), dtype='float64')
    NMSE_g = np.empty((max_iter, 21), dtype='float64')
    NMSE_gh = np.empty((max_iter, 21), dtype='float64')

    for iter in range(0, max_iter):
      filename_train = '/content/drive/My Drive/path_to_your_folder/train_data/ch_est_data_' + 'M_' + str(M) + '_N_' + str(N) + '_K_10_snr_-30_0.mat'

      data = sio.loadmat(filename_train)

      A = data["A"]
      X = data["X"]
      E = data["E"]
      A = np.ones(A.shape)
      y = data["y"]

      n_classes = y.shape[-1]  # Number of classes


      # Parameters
      P = X.shape[-2]       # Number of nodes in the graphs
      F = X[0].shape[-1]    # Dimension of node features
      S = E[0].shape[-1]    # Dimension of edge features
      n_out = y.shape[-1]   # Dimension of the target

      # Train/test split
      A_train, A_test, \
      X_train, X_test, \
      E_train, E_test, \
      y_train, y_test = train_test_split(A, X, E, y, test_size=0.2, random_state=0)

      ################################################################################
      # BUILD MODEL
      ################################################################################
      X_in = Input(shape=(P, F))
      A_in = Input(shape=(P, P))
      E_in = Input(shape=(P, P, S))


      gc1 = GATConv(128, activation='relu', kernel_regularizer=l2(l2_reg))([X_in, A_in])
      gc2 = GATConv(32, activation='relu', kernel_regularizer=l2(l2_reg))([gc1, A_in])
      pool = GlobalAttentionPool(128)(gc2)

      output = Dense(n_classes)(pool)

      # Build model
      model = Model(inputs=[X_in, A_in], outputs=output)
      optimizer = Adam(lr=learning_rate)
      model.compile(optimizer=optimizer, loss='mse')
      model.summary()

      # Train model
      estimator = model.fit([X_train, A_train],
                y_train,
                batch_size=batch_size,
                validation_split=0.1,
                epochs=epochs,
                callbacks=[
                    EarlyStopping(patience=es_patience, restore_best_weights=True)
                ])

      # Evaluate model
      print('Evaluating model.')
      eval_results = model.evaluate([X_test, A_test],
                                    y_test,
                                    batch_size=batch_size)

      filename_val_loss = '/content/drive/My Drive/path_to_your_folder/gat_self_loop/loss/val_loss_' + 'M_' + str(M) + '_N_' + str(N) + '_K_' + str(K) + '.txt'
      with open(filename_val_loss, 'wb') as f:
          np.savetxt(f, np.array(estimator.history['val_loss']), delimiter=',')

      filename_loss = '/content/drive/My Drive/path_to_your_folder/results/gat_self_loop/loss/loss_' + 'M_' + str(M) + '_N_' + str(N) + '_K_' + str(K) + '.txt'
      with open(filename_loss, 'wb') as f:
          np.savetxt(f, np.array(estimator.history['loss']), delimiter=',')

      ################################################################################
      # TEST MODEL
      ################################################################################
      
      idx = 0

      for snr in range(-30, 12, 2):
        filename_test = '/content/drive/My Drive/path_to_your_folder/test_data/ch_est_data_' + 'M_' + str(M) + '_N_' + str(N) + '_K_10_snr_' + str(snr) + '.mat'
        data = sio.loadmat(filename_test)

        A = data["A"]
        X = data["X"]
        E = data["E"]
        A = np.ones(A.shape)
        y = data["y"]

        # Parameters
        P = X.shape[-2]       # Number of nodes in the graphs
        F = X[0].shape[-1]    # Dimension of node features
        S = E[0].shape[-1]    # Dimension of edge features
        n_out = y.shape[-1]   # Dimension of the target

        MM = int(M/2)

        y_est = model.predict([X, A])
        nmse_h = (np.sum((y_est[:, 0:2*M] - y[:, 0, 0:2*M])**2)/(len(y_est[:, 0:2*M])*y_est[:, 0:2*M].shape[-1]))/np.sum(y[:, 0, 0:2*M]**2)
        nmse_g = (np.sum((y_est[:, 2*M:4*M] - y[:, 0, 2*M:4*M])**2)/(len(y_est[:, 2*M:4*M])*y_est[:, 2*M:4*M].shape[-1]))/np.sum(y[:, 0, 2*M:4*M]**2)
        nmse_gh = (np.sum((y_est - y[:, 0, :])**2)/(len(y_est)*y_est.shape[-1]))/np.sum(y**2)
        print(nmse_h, nmse_g, nmse_gh)

        NMSE_h[iter, idx] = nmse_h
        NMSE_g[iter, idx] = nmse_g
        NMSE_gh[iter, idx] = nmse_gh
        idx = idx + 1

        h_est = y_est 

        filename_nmse_gh = '/content/drive/My Drive/path_to_your_folder/results/gat/channel_est/lower_N/channel_est_' + 'M_' + str(M) + '_N_' + str(N) + '_K_' + str(K) + '_snr_' + str(snr) + '_' + str(iter) + '.txt'
        with open(filename_nmse_gh, 'wb') as f:
          np.savetxt(f, h_est, delimiter=',')

      filename_nmse_h = '/content/drive/My Drive/path_to_your_folder/results/gat/nmse_h_' + 'M_' + str(M) + '_N_' + str(N) + '_K_' + str(K) + '_snr_' + str(snr) + '.txt'
      with open(filename_nmse_h, 'wb') as f:
        np.savetxt(f, np.array(np.mean(NMSE_h, axis = 0)), delimiter=',')

      filename_nmse_g = '/content/drive/My Drive/path_to_your_folder/results/gat/nmse_g_' + 'M_' + str(M) + '_N_' + str(N) + '_K_' + str(K) + '_snr_' + str(snr) + '.txt'
      with open(filename_nmse_g, 'wb') as f:
        np.savetxt(f, np.array(np.mean(NMSE_g, axis = 0)), delimiter=',')
      
      filename_nmse_gh = '/content/drive/My Drive/path_to_your_folder/results/gat/nmse_gh_' + 'M_' + str(M) + '_N_' + str(N) + '_K_' + str(K) + '_snr_' + str(snr) + '.txt'
      with open(filename_nmse_gh, 'wb') as f:
        np.savetxt(f, np.array(np.mean(NMSE_gh, axis = 0)), delimiter=',')

      
        